In [1]:
import pandas as pd
import os
from erddapy import ERDDAP
import numpy as np

print("Loading Dataset...")
if os.path.exists("dataset.csv"):
    df = pd.read_csv("dataset.csv")
else:
    e = ERDDAP(server="https://dev.cioosatlantic.ca/erddap",
                 protocol="tabledap", )
    e.auth = ("cioosatlantic", "4oceans")
    e.response = "csv"
    IDlist = ['wpsu-7fer','knwz-4bap','eb3n-uxcb','x9dy-aai9',
              'a9za-3t63','eda5-aubu','adpu-nyt8','v6sa-tiit','mq2k-54s4','9qw2-yb2f']
    list = []
    for val in IDlist:
        e.dataset_id = val
        df = e.to_pandas()
        list.append(df)
        print(val, "loaded")
        
    df = pd.concat(list, axis=0, ignore_index=True) 
    print("dataframe created")
    df.to_csv("dataset.csv")
print("Dataset loaded.")
df = df.set_index(df['time (UTC)'].astype(np.datetime64))


Loading Dataset...
wpsu-7fer loaded
knwz-4bap loaded
eb3n-uxcb loaded
x9dy-aai9 loaded
a9za-3t63 loaded
eda5-aubu loaded
adpu-nyt8 loaded
v6sa-tiit loaded
mq2k-54s4 loaded
9qw2-yb2f loaded
dataframe created
Dataset loaded.


In [ ]:
import panel as pn
import datashader as ds
import holoviews as hv
import holoviews.operation.datashader as hd
from datashader.colors import viridis

pn.extension()

stationlist = list(sorted(df["waterbody_station"].unique()))
station_selector = pn.widgets.Select(name="Station", options=stationlist, value = stationlist[0])


def tempScatterPlot(station):

    print("Creating new temp plot")
    print("filtering...")
    data = df[df.waterbody_station == station]
    #data['time (UTC)'] = pd.to_datetime(data['time (UTC)'],
                                       format="%Y-%m-%dT%H:%M:%SZ")
    shaded = hd.datashade(hv.Points(data, ['time (UTC)', 'Temperature (degrees Celsius)']).opts(),
                      cmap=viridis, aggregator=ds.count('depth (m)'))
    tempplot = hd.dynspread(shaded, threshold=.999, max_px=2).opts(bgcolor='white',
                                                   width=900, height=500)
    return tempplot

def oxygenPlot(station):
    data = df[df.waterbody_station == station]
    shaded = hd.datashade(hv.Points(data, ['time (UTC)', 'Dissolved_Oxygen (% saturation)']).opts(),
                      cmap=viridis, aggregator=ds.count('depth (m)'))
    oxyplot = hd.dynspread(shaded, threshold = 0.999, max_px=2).opts(bgcolor='white',
                                                   width=900, height=500)
    
    return oxyplot



temppane = pn.pane.HoloViews(tempScatterPlot(station_selector.value))
oxypane = pn.pane.HoloViews(oxygenPlot(station_selector.value))

def tempstation(target, event):
    target.object = tempScatterPlot(event.new)
def oxystation(target, event):
    target.object = oxygenPlot(event.new)

station_selector.link(temppane, callbacks={"value":tempstation})
station_selector.link(oxypane, callbacks={"value":oxystation})

widgets = pn.Column(pn.Spacer(height=0), station_selector, width=400)
charts = pn.Column(temppane, pn.Spacer(height=0), oxypane, pn.Spacer(height=0))
dashboard = pn.Row(charts, pn.Spacer(width=0), widgets)
dashboard


# data = df.drop_duplicates(subset='waterbody_station')


In [3]:
examplestation = df[df.waterbody_station== "Green Harbour-Blue Island"]
examplestation['time (UTC)'] = pd.to_datetime(examplestation['time (UTC)'],
                                       format="%Y-%m-%dT%H:%M:%SZ")
import panel as pn
pn.extension()
import datashader as ds
import holoviews as hv
import holoviews.operation.datashader as hd
from datashader.colors import viridis

shaded = hd.datashade(hv.Points(examplestation, ['time (UTC)', 'Temperature (degrees Celsius)']).opts(),
                      cmap=viridis, aggregator=ds.count('depth (m)'))
plot = hd.dynspread(shaded, threshold=.999, max_px=2).opts(bgcolor='white',
                                                   width=900, height=500)
title = pn.pane.Markdown('Green Harbour-Blue Island')
plotpane = pn.pane.HoloViews(plot)
dashboard = pn.Column(title, plotpane)
dashboard

C:\Users\rinez\AppData\Local\Temp/ipykernel_25440/3875394634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  examplestation['time (UTC)'] = pd.to_datetime(examplestation['time (UTC)'],


Column
    [0] Markdown(str)
    [1] HoloViews(DynamicMap)